In [6]:
import os
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

from llama_index.agent.openai import OpenAIAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.prompts.system import DEFAULT

from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool
from openai.types.chat import ChatCompletionMessageToolCall

import nest_asyncio

nest_asyncio.apply()

In [7]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)
def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

In [9]:

llm = OpenAI(model="gpt-3.5-turbo")
agent = OpenAIAgent.from_tools(
    [multiply_tool, add_tool], llm=llm, verbose=True
)
response = agent.chat("What is (121 * 3) + 42?")
print(str(response))

Added user message to memory: What is (121 * 3) + 42?
=== Calling Function ===
Calling function: multiply with args: {"a": 121, "b": 3}
Got output: 363

=== Calling Function ===
Calling function: add with args: {"a": 363, "b": 42}
Got output: 405

The result of (121 * 3) + 42 is 405.


In [14]:
import wikipedia
from pydantic import BaseModel
from llama_index.core.tools import FunctionTool
from llama_index.llms.openai import OpenAI
from typing import Dict

# class WikiSearchResult(BaseModel):
#     title: str
#     url: str

# class WikiArticle(BaseModel):
#     title: str
#     content: str
#     url: str

def wikipedia_similar_articles(query: str) -> list[Dict[str,str]]:
    """Search Wikipedia for articles similar to the given query and return titles and URLs."""
    search_results = wikipedia.search(query, results=5)
    result_list = []
    for result in search_results:
        try:
            page = wikipedia.page(result)
            result_list.append({"title": page.title, "url": page.url})
        except wikipedia.exceptions.DisambiguationError as e:
            # Handle disambiguation pages by logging or ignoring
            print(f"Disambiguation page: {e.options}")
        except wikipedia.exceptions.PageError:
            print(f"PageError: {result}")
    return result_list



def wikipedia_full_article(query: str) -> Dict[str,str]:
    """Fetch the full Wikipedia article for the given query."""
    try:
        page = wikipedia.page(query)
        return {"title": page.title, "content": page.content, "url": page.url}
    except wikipedia.exceptions.DisambiguationError as e:
        # Handle disambiguation pages
        print(f"Disambiguation page: {e.options}")
    except wikipedia.exceptions.PageError:
        print(f"PageError: {query}")
    return None

# Wrap these functions in a tool
similar_articles_tool = FunctionTool.from_defaults(fn=wikipedia_similar_articles)
full_article_tool = FunctionTool.from_defaults(fn=wikipedia_full_article)

In [15]:

llm = OpenAI(model="gpt-3.5-turbo")

agent = OpenAIAgent.from_tools(
    [similar_articles_tool, full_article_tool],
    llm=llm,
    verbose=True,
    system_prompt=DEFAULT,
)

In [16]:

response = agent.chat("Wassssssup")
print(response)

Added user message to memory: Wassssssup
Hello! How can I assist you today?


In [17]:
response = agent.chat("Tell me about the Eiffel Tower")
print(response)

Added user message to memory: Tell me about the Eiffel Tower
=== Calling Function ===
Calling function: wikipedia_full_article with args: {"query":"Eiffel Tower"}
Got output: {'title': 'Eiffel Tower', 'content': 'The Eiffel Tower (  EYE-fəl; French: Tour Eiffel [tuʁ ɛfɛl] ) is a wrought-iron lattice tower on the Champ de Mars in Paris, France. It is named after the engineer Gustave Eiffel, whose company designed and built the tower from 1887 to 1889.\nLocally nicknamed "La dame de fer" (French for "Iron Lady"), it was constructed as the centerpiece of the 1889 World\'s Fair, and to crown the centennial anniversary of the French Revolution. Although initially criticised by some of France\'s leading artists and intellectuals for its design, it has since become a global cultural icon of France and one of the most recognisable structures in the world.  The tower received 5,889,000 visitors in 2022. The Eiffel Tower is the most visited monument with an entrance fee in the world: 6.91 millio

In [18]:
response = agent.chat("Give me some articles similar to the Eiffel Tower")
print(response)

Added user message to memory: Give me some articles similar to the Eiffel Tower
=== Calling Function ===
Calling function: wikipedia_similar_articles with args: {"query":"Eiffel Tower"}


/mnt/c/Users/mwp89/Desktop/envs/wiki_search_env/lib/python3.9/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /mnt/c/Users/mwp89/Desktop/envs/wiki_search_env/lib/python3.9/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Disambiguation page: ['Eiffel Tower (Paris, Tennessee)', 'Eiffel Tower (Paris, Texas)', 'Eiffel Tower (Six Flags)', 'at Paris Las Vegas', 'Tour Eiffel Bridge', 'Eiffel Tower (Delaunay series)', 'EiffelTowers Nijmegen', 'Heroes Den Bosch', 'Tour Eiffel (disambiguation)', 'List of Eiffel Tower replicas', 'Eiffel (disambiguation)', 'Tower (disambiguation)', 'All pages with titles beginning with Eiffel Tower', 'All pages with titles containing Eiffel Tower']
Got output: [{'title': 'Eiffel Tower', 'url': 'https://en.wikipedia.org/wiki/Eiffel_Tower'}, {'title': 'Eiffel Tower replicas and derivatives', 'url': 'https://en.wikipedia.org/wiki/Eiffel_Tower_replicas_and_derivatives'}, {'title': 'List of the 72 names on the Eiffel Tower', 'url': 'https://en.wikipedia.org/wiki/List_of_the_72_names_on_the_Eiffel_Tower'}, {'title': 'Gustave Eiffel', 'url': 'https://en.wikipedia.org/wiki/Gustave_Eiffel'}]

Here are some articles similar to the Eiffel Tower that you may find interesting:

1. [Eiffel Tow

In [19]:
response = agent.chat("the school of athens")
print(response)

Added user message to memory: the school of athens
=== Calling Function ===
Calling function: wikipedia_similar_articles with args: {"query":"The School of Athens"}
Disambiguation page: ['Athens City Schools (Alabama)', 'Athens City Schools (Tennessee)']
Got output: [{'title': 'The School of Athens', 'url': 'https://en.wikipedia.org/wiki/The_School_of_Athens'}, {'title': 'British School at Athens', 'url': 'https://en.wikipedia.org/wiki/British_School_at_Athens'}, {'title': 'Athens Academy (school)', 'url': 'https://en.wikipedia.org/wiki/Athens_Academy_(school)'}, {'title': 'Athens High School (Ohio)', 'url': 'https://en.wikipedia.org/wiki/Athens_High_School_(Ohio)'}]

Here are some articles related to "The School of Athens":

1. [The School of Athens](https://en.wikipedia.org/wiki/The_School_of_Athens) - A famous fresco painting by Raphael depicting a gathering of philosophers and scholars in ancient Greece.
2. [British School at Athens](https://en.wikipedia.org/wiki/British_School_at_

In [20]:
response = agent.chat("Who scored the most goals in the European Champion's League in 2020?")
print(response)

Added user message to memory: Who scored the most goals in the European Champion's League in 2020?
=== Calling Function ===
Calling function: wikipedia_full_article with args: {"query":"UEFA Champions League top scorers 2020"}
Got output: {'title': 'List of UEFA Champions League top scorers', 'content': "The UEFA Champions League, known until 1992 as the European Champion Clubs' Cup or colloquially as the European Cup, is an annual association football cup competition organised by UEFA since 1955. Originally a straight knockout competition open only to champion clubs, the tournament was expanded during the 1990s to incorporate a round-robin group phase and more teams. The expansion resulted in more games being played, increasing players' goalscoring chances; thus the rankings are weighted in favour of modern players: only seven out of the 52 players on the list never competed in the reformed Champions League.\nWith 140 goals, Cristiano Ronaldo is currently the all-time top scorer in th

In [21]:
response = agent.chat("Who is the president?")
print(response)

Added user message to memory: Who is the president?
I don't have real-time information on current events. To get the most up-to-date information on the current president, I recommend checking a reliable news source or conducting a quick search online for the latest updates. If you have any other questions or need assistance with something else, feel free to let me know!


In [24]:
prompt = agent.get_prompts()
for k, v in prompt.items():
    print(f"Prompt: {k}\n\nValue: {v.template}")